In [1]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 103.9 MB/s eta 0:00:00


In [2]:
import pdfplumber
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import numpy as np

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
books = ['/content/Deep+Learning+Ian+Goodfellow.pdf', '/content/Free PDF - Generative AI Foundations in Python.pdf',
         '/content/asset-v1_ColumbiaX+CSMM.101x+1T2017+type@asset+block@AI_edx_ml_5.1intro.pdf',
         '/content/deep-learning-material-dept-ece-ase-blr-1.pdf','thebook.pdf', '/content/mlops.pdf']

In [7]:
full_text = ''

for book in books:
  with pdfplumber.open(book) as pdf:
    for page in pdf.pages:
      full_text = full_text + page.extract_text() + '\n'
    print(book, 'is read completely')

/content/Deep+Learning+Ian+Goodfellow.pdf is read completely
/content/Free PDF - Generative AI Foundations in Python.pdf is read completely
/content/asset-v1_ColumbiaX+CSMM.101x+1T2017+type@asset+block@AI_edx_ml_5.1intro.pdf is read completely
/content/deep-learning-material-dept-ece-ase-blr-1.pdf is read completely
thebook.pdf is read completely


/content/mlops.pdf is read completely


In [8]:
full_text = full_text.lower()

In [9]:
len(full_text)

3841861

In [10]:
tokens = word_tokenize(full_text)

In [11]:
vocab = {'<UNK>' : 0}

In [12]:
for token in set(tokens):
  if token not in vocab:
    vocab[token] = len(vocab)

In [13]:
len(vocab)

28321

In [14]:
input_sentence = full_text.split('\n')

In [15]:
def text_to_indices(text, vocab = vocab):
  numerical_text = []
  for token in text:
    if token in vocab:
      numerical_text.append(vocab[token])
    else:
      numerical_text.append(vocab['<UNK>'])

  return numerical_text

In [16]:
input_numerical_sentence = []

for sentence in input_sentence:
  input_numerical_sentence.append(text_to_indices(word_tokenize(sentence)))

In [17]:
sentence_to_remove = []

for i in range(len(input_numerical_sentence)):
  if len(input_numerical_sentence[i]) < 2:
      sentence_to_remove.append(input_numerical_sentence[i])

In [18]:
for sentence in sentence_to_remove:
  input_numerical_sentence.remove(sentence)

In [19]:
training_sequence = []

for sentence in input_numerical_sentence:
  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [20]:
list_len = []

for sentence in training_sequence:
  list_len.append(len(sentence))

In [21]:
from collections import Counter
import math

In [22]:
# if i do the zero padding while taking 696 as my largest sentence, then i will have alot of zeros in padding causing computational complexity

Counter(list_len)

Counter({2: 61292,
         3: 57581,
         4: 54575,
         5: 51735,
         6: 49499,
         7: 47458,
         8: 45826,
         9: 44119,
         10: 42473,
         11: 40551,
         12: 37866,
         13: 33853,
         14: 28890,
         15: 23187,
         16: 17701,
         17: 13088,
         18: 9598,
         19: 6991,
         20: 5189,
         21: 3912,
         22: 3050,
         23: 2349,
         24: 1826,
         25: 1431,
         26: 1149,
         27: 948,
         28: 795,
         29: 660,
         30: 562,
         31: 471,
         32: 409,
         33: 348,
         34: 297,
         35: 251,
         36: 216,
         37: 183,
         38: 167,
         39: 153,
         40: 138,
         41: 126,
         42: 116,
         43: 110,
         44: 96,
         45: 86,
         46: 83,
         47: 77,
         48: 76,
         49: 70,
         50: 59,
         51: 58,
         52: 51,
         53: 45,
         54: 41,
         55: 34,
       

In [23]:
split_list = []
remove_large_sentence = []

for i in training_sequence[:]:
  if len(i) < 50:
    continue
  else:
    strt = 0
    end = 50
    remove_large_sentence.append(i)
    for j in range(math.ceil(len(i)/50)):
      sub_list = i[strt: end]
      if len(sub_list) > 2:
        split_list.append(sub_list)
        strt = end
        end = end + 50

In [24]:
Counter([len(i) for i in split_list])

Counter({50: 9777,
         3: 80,
         4: 76,
         5: 69,
         6: 67,
         7: 63,
         8: 61,
         9: 60,
         10: 59,
         11: 58,
         12: 57,
         13: 57,
         14: 56,
         15: 56,
         16: 55,
         17: 55,
         18: 54,
         19: 53,
         20: 52,
         21: 50,
         22: 48,
         23: 48,
         24: 45,
         25: 43,
         26: 42,
         27: 42,
         28: 41,
         29: 41,
         30: 41,
         31: 41,
         32: 41,
         33: 41,
         34: 40,
         35: 40,
         36: 38,
         37: 38,
         38: 38,
         39: 38,
         40: 38,
         41: 36,
         42: 36,
         43: 36,
         44: 36,
         45: 36,
         46: 36,
         47: 36,
         48: 36,
         49: 36})

In [25]:
for i in remove_large_sentence:
  training_sequence.remove(i)

In [26]:
for i in split_list:
  training_sequence.append(i)

In [27]:
# now all the sentence have maximum word of 50.
Counter([len(i) for i in training_sequence])

Counter({2: 61292,
         3: 57661,
         4: 54651,
         5: 51804,
         6: 49566,
         7: 47521,
         8: 45887,
         9: 44179,
         10: 42532,
         11: 40609,
         12: 37923,
         13: 33910,
         14: 28946,
         15: 23243,
         16: 17756,
         17: 13143,
         18: 9652,
         19: 7044,
         20: 5241,
         21: 3962,
         22: 3098,
         23: 2397,
         24: 1871,
         25: 1474,
         26: 1191,
         27: 990,
         28: 836,
         29: 701,
         30: 603,
         31: 512,
         32: 450,
         33: 389,
         34: 337,
         35: 291,
         36: 254,
         37: 221,
         38: 205,
         39: 191,
         40: 176,
         41: 162,
         42: 152,
         43: 146,
         44: 132,
         45: 122,
         46: 119,
         47: 113,
         48: 112,
         49: 106,
         50: 9777})

In [28]:
# making all the sentence of equal length
training_sequence = pad_sequences(training_sequence)

In [29]:
training_sequence = torch.tensor(training_sequence)

In [30]:
x = training_sequence[:, :-1]

In [31]:
y = training_sequence[:, -1]

In [32]:
class CustomDataset(Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __len__(self):
    return len(self.x)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [33]:
dataset = CustomDataset(x,y)

In [34]:
dataloader = DataLoader(dataset, batch_size=100, shuffle=True, num_workers=2, pin_memory=True)

In [35]:
class GeekLSTM(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 200)
    self.lstm = nn.LSTM(200, 256, batch_first=True)
    self.fc = nn.Linear(256, vocab_size)

  def forward(self, x):
    embeddings = self.embedding(x)
    intermidiate_hidden_state, (final_hidden_state, final_cell_state) = self.lstm(embeddings)
    out = self.fc(final_hidden_state.squeeze(0))
    return out

#### understanding flow of tensors

In [36]:
for batchx, batchy in dataloader:
  print(batchx.shape,batchy.shape)
  break

torch.Size([100, 49]) torch.Size([100])


In [37]:
batchx.shape # batch of 100 sentence, each sentence has 50 words (0-49)

torch.Size([100, 49])

In [38]:
x = nn.Embedding(len(vocab), 200)

xout = x(batchx)

# batch of 100 sentence, each sentence has 50 words, and each word is represented using vector embedding of 200 numbers
xout.shape

torch.Size([100, 49, 200])

In [39]:
y = nn.LSTM(200, 256, batch_first=True)

yout = y(xout)[1][0].squeeze(0) # need to squeeze one dim

# first word of each 100 sentence, in 256-D LSTM output
yout.shape

torch.Size([100, 256])

In [40]:
z = nn.Linear(256, len(vocab))

zout = z(yout)

# first word of each 100 sentence, in 28320 dim (num of unique sentence)
zout.shape

torch.Size([100, 28321])

In [41]:
model = GeekLSTM(len(vocab))
model = model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
epochs = 50
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [69]:
model.train()
for epoch in range(10):
  epoch_loss = []
  for batchx, batchy in dataloader:
    batchx, batchy = batchx.to(device), batchy.to(device)
    output = model(batchx)
    loss = loss_function(output, batchy.long())

    with torch.no_grad():
      epoch_loss.append(loss.item())

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  print(f'in epoch {epoch} loss is {np.mean(epoch_loss)}')

In [70]:
def predict(model, text, vocab = vocab):
  model.eval()
  with torch.no_grad():
    text = text.lower()
    numerical_text = text_to_indices(word_tokenize(text))
    numerical_text = torch.tensor(numerical_text).unsqueeze(0)
    numerical_text = numerical_text.to(device)
    output = model(numerical_text)
    output = torch.softmax(output, dim=1)
    idx = output.argmax(dim=1)
    return text + ' ' + list(vocab.keys())[idx]

In [79]:
predict_token = 5

input_text = 'normal distribution'

for i in range(predict_token):
  out = predict(model, input_text)
  print(out)
  input_text = out

normal distribution with
normal distribution with mean
normal distribution with mean and
normal distribution with mean and variance
normal distribution with mean and variance x.


In [82]:
checkpoint = {
    'epoch': epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
}

torch.save(checkpoint, 'checkpoint.pth')
print("Checkpoint saved!")

Checkpoint saved!
